# 读取表格所需数据

In [87]:
import time
import xlrd
import re
#打开科目余额表.xls工作簿
data = xlrd.open_workbook('科目余额表.xls') # 打开xls文件
#通过名称sheet1获取表格数据
table = data.sheet_by_name('Sheet')


# 按列数读取，读取第0列
# 都从第四行开始读取
first_col = table.col_values(0)
a = first_col[6:]
#a

# 按列数读取，读取第1列
s = table.col_values(1)
b = s[6:]
#b

# 按列数读取，读取第倒数第2列
t = table.col_values(-1)
c = t[6:]
#c


from pandas.core.frame import DataFrame
diction={"a":a,"b":b,"c":c}#将列表a，b转换成字典
df=DataFrame(diction)#将字典转换成为数据框
#print(df)

time.sleep(1)

WARNING *** file size (141438) not 512 + multiple of sector size (512)


In [88]:
# a

# 对于第0列数据（即编号），要找到不是字符串的数字并且改成字符串

In [89]:
zfc=[]
# 查找哪一个不是字符型变量
for i in range(df.shape[0]):
    #print(len(str(df.iloc[i,0])))
    if type(df.iloc[i,0])!=str:
        zfc.append(i)
for i in zfc:
    df.iloc[i,0]=str(int(df.iloc[i,0]))
    
# 找出所有的7位数总行的索引
# 索引存储的列表
bianhao7 = []
for i in range(df.shape[0]):
    if len(df.iloc[i,0])==7:
        bianhao7.append(i)
#bianhao7
# 找出所有的10位数总行的索引
# 索引存储的列表
bianhao10 = []
for i in range(df.shape[0]):
    if len(df.iloc[i,0])==10:
        bianhao10.append(i)

df7 = df.iloc[bianhao7,:]
df10 = df.iloc[bianhao10,:]

# 索引重新排序
df71=df7.reset_index(drop=True)
#print(df71)
df10.head()
df101=df10.reset_index(drop=True)
#print(df101)

time.sleep(1)

# 形成所需数据的数据框

In [90]:
# import re
# aaa = 'assxsxs12344'
# cop = re.compile("[^0-9]") # 匹配不是中文、大小写、数字的其他字符
# string1 = cop.sub('', aaa) #将string1中匹配到的字符替换成空字符
# string1

In [91]:
# 文字生成 银行-分行
shiyan=df101
for i in range(df71.shape[0]):
    for j in range(shiyan.shape[0]):
        if df71.iloc[i,0]==shiyan.iloc[j,0][:7]:
            shiyan.iloc[j,1]=df71.iloc[i,1]+'-'+shiyan.iloc[j,1]

shiyan1=shiyan
# 增加一列
shiyan1['add_column']  = 1
shiyan1['add_column1']  = 1
#最后一列账号编号
for i in range(shiyan1.shape[0]):
    aaa = shiyan1.iloc[i,1]
    cop = re.compile("[^0-9]") # 匹配不是中文、大小写、数字的其他字符
    string1 = cop.sub('', aaa) #将string1中匹配到的字符替换成空字符
#     shiyan1.iloc[i,3]="\'"+string1+"\'"
    shiyan1.iloc[i,4]=string1
    
for i in range(shiyan1.shape[0]):
    shiyan1.iloc[i,3]=shiyan1.iloc[i,1][-4:]
#shiyan1
# 字符串去数字

import string
for i in range(shiyan1.shape[0]):
    shiyan1.iloc[i,1]=shiyan1.iloc[i,1].rstrip(string.digits)
    
    
    
# 金额完善
for i in range(shiyan1.shape[0]):
    # 由于二进制存储误差用round(,0)四舍五入到整数位
    shiyan1.iloc[i,2]='¥'+str(int(round(shiyan1.iloc[i,2]*100,0)))
    
time.sleep(1)

# 打印信息表,导出信息表格

In [96]:
# shiyan1.head()

In [97]:
shiyan1.head()
# shiyan1["add_column"] = shiyan1["add_column"].astype(str)

,a,b,c,add_column,add_column1
0,1002001001,中国工商银行-上海第一支行,¥33401795,7603,10012935293039977603
1,1002001002,中国工商银行-沈阳文化路支行,¥18656012,3337,3301003519249113337
2,1002001006,中国工商银行-张江川沙第二支行,¥987755616,2339,1001711819006862339
3,1002001009,中国工商银行-珠海夏湾支行,¥6325593,0779,2002025638000000779
4,1002001010,中国工商银行-珠海夏湾支行,¥101363285,1034,2002025638000001034


In [94]:
#shiyan1

In [95]:
# 输出shiyan1 表格
shiyan1.to_csv('shiyan1.csv')

# 文档打开地址得改改

In [98]:
from xlutils.copy import copy
import xlrd
import xlwt
# 保持格式不变
tem_excel = xlrd.open_workbook(r'余额调节表.xls'
                              ,formatting_info=True)
# 拷贝工作簿
new_excel = copy(tem_excel)

# 设计单元格格式

In [99]:
# 开户银行和账号格式 style0
style0 = xlwt.XFStyle()

font = xlwt.Font()
font.name = '宋体'
#加粗与否
font.bold = False
# 前一个数字是字号
font.height = 14*20 
# 样式复制
style0.font = font


# 对齐
alignment = xlwt.Alignment()
# 水平方向
alignment.horz = xlwt.Alignment.HORZ_CENTER
# 垂直方向
alignment.vert = xlwt.Alignment.VERT_CENTER
style0.alignment = alignment


# 上面两个金额数字单元格的格式

In [100]:
# 上面两个金额的格式
style1 = xlwt.XFStyle()

font = xlwt.Font()
font.name = '宋体'
font.bold = False
# 前一个数字是字号
font.height = 10*20 
# 样式复制
style1.font = font

# 单元格边框
borders = xlwt.Borders()
borders.top = xlwt.Borders.MEDIUM
borders.bottom = xlwt.Borders.MEDIUM
borders.left = xlwt.Borders.THIN
borders.right = xlwt.Borders.THICK
style1.borders = borders

# 对齐
alignment = xlwt.Alignment()
# 水平方向
alignment.horz = xlwt.Alignment.HORZ_CENTER
# 垂直方向
alignment.vert = xlwt.Alignment.VERT_CENTER
style1.alignment = alignment

In [101]:
style2 = xlwt.XFStyle()

font = xlwt.Font()
font.name = '宋体'
font.bold = False
# 前一个数字是字号
font.height = 10*20 
# 样式复制
style2.font = font

# 单元格边框
borders = xlwt.Borders()
borders.top = xlwt.Borders.MEDIUM
borders.bottom = xlwt.Borders.MEDIUM
borders.left = xlwt.Borders.MEDIUM
borders.right = xlwt.Borders.THIN
style2.borders = borders

# 对齐
alignment = xlwt.Alignment()
# 水平方向
alignment.horz = xlwt.Alignment.HORZ_CENTER
# 垂直方向
alignment.vert = xlwt.Alignment.VERT_CENTER
style2.alignment = alignment

In [102]:
style3 = xlwt.XFStyle()

font = xlwt.Font()
font.name = '宋体'
font.bold = False
# 前一个数字是字号
font.height = 10*20 
# 样式复制
style3.font = font

# 单元格边框
borders = xlwt.Borders()
borders.top = xlwt.Borders.MEDIUM
borders.bottom = xlwt.Borders.MEDIUM
borders.left = xlwt.Borders.THIN
borders.right = xlwt.Borders.MEDIUM
style3.borders = borders

# 对齐
alignment = xlwt.Alignment()
# 水平方向
alignment.horz = xlwt.Alignment.HORZ_CENTER
# 垂直方向
alignment.vert = xlwt.Alignment.VERT_CENTER
style3.alignment = alignment

In [103]:
# 初始化空样式
style4 = xlwt.XFStyle()

font = xlwt.Font()
font.name = '宋体'
font.bold = False
# 前一个数字是字号
font.height = 10*20 
# 样式复制
style4.font = font

# 单元格边框
borders = xlwt.Borders()
borders.top = xlwt.Borders.MEDIUM
borders.bottom = xlwt.Borders.MEDIUM
borders.left = xlwt.Borders.THIN
borders.right = xlwt.Borders.THIN
style4.borders = borders

# 对齐
alignment = xlwt.Alignment()
# 水平方向
alignment.horz = xlwt.Alignment.HORZ_CENTER
# 垂直方向
alignment.vert = xlwt.Alignment.VERT_CENTER
style4.alignment = alignment

In [104]:
stylelist=[style1,style2,style3,style4,style2,style3,style4,style2,style3,style4,style2]

# 下面两个金额数字单元格的格式

In [105]:
style7 = xlwt.XFStyle()

font = xlwt.Font()
font.name = '宋体'
font.bold = False
# 前一个数字是字号
font.height = 10*20 
# 样式复制
style7.font = font

# 单元格边框
borders = xlwt.Borders()
borders.top = xlwt.Borders.MEDIUM
borders.bottom = xlwt.Borders.THICK
borders.left = xlwt.Borders.THIN
borders.right = xlwt.Borders.THICK
style7.borders = borders

# 对齐
alignment = xlwt.Alignment()
# 水平方向
alignment.horz = xlwt.Alignment.HORZ_CENTER
# 垂直方向
alignment.vert = xlwt.Alignment.VERT_CENTER
style7.alignment = alignment

In [106]:
style8 = xlwt.XFStyle()

font = xlwt.Font()
font.name = '宋体'
font.bold = False
# 前一个数字是字号
font.height = 10*20 
# 样式复制
style8.font = font

# 单元格边框
borders = xlwt.Borders()
borders.top = xlwt.Borders.MEDIUM
borders.bottom = xlwt.Borders.THICK
borders.left = xlwt.Borders.MEDIUM
borders.right = xlwt.Borders.THIN
style8.borders = borders

# 对齐
alignment = xlwt.Alignment()
# 水平方向
alignment.horz = xlwt.Alignment.HORZ_CENTER
# 垂直方向
alignment.vert = xlwt.Alignment.VERT_CENTER
style8.alignment = alignment

In [107]:
style9 = xlwt.XFStyle()

font = xlwt.Font()
font.name = '宋体'
font.bold = False
# 前一个数字是字号
font.height = 10*20 
# 样式复制
style9.font = font

# 单元格边框
borders = xlwt.Borders()
borders.top = xlwt.Borders.MEDIUM
borders.bottom = xlwt.Borders.THICK
borders.left = xlwt.Borders.THIN
borders.right = xlwt.Borders.MEDIUM
style9.borders = borders

# 对齐
alignment = xlwt.Alignment()
# 水平方向
alignment.horz = xlwt.Alignment.HORZ_CENTER
# 垂直方向
alignment.vert = xlwt.Alignment.VERT_CENTER
style9.alignment = alignment

In [108]:
# 初始化空样式
style10 = xlwt.XFStyle()

font = xlwt.Font()
font.name = '宋体'
font.bold = False
# 前一个数字是字号
font.height = 10*20 
# 样式复制
style10.font = font

# 单元格边框
borders = xlwt.Borders()
borders.top = xlwt.Borders.MEDIUM
borders.bottom = xlwt.Borders.THICK
borders.left = xlwt.Borders.THIN
borders.right = xlwt.Borders.THIN
style10.borders = borders

# 对齐
alignment = xlwt.Alignment()
# 水平方向
alignment.horz = xlwt.Alignment.HORZ_CENTER
# 垂直方向
alignment.vert = xlwt.Alignment.VERT_CENTER
style10.alignment = alignment

In [109]:
stylelist1=[style7,style8,style9,style10,style8,style9,style10,style8,style9,style10,style8]

In [110]:
# 金额文字单元格样式
style5 = xlwt.XFStyle()

font = xlwt.Font()
font.name = '宋体'
font.bold = False
# 前一个数字是字号
font.height = 12*20 
# 样式复制
style5.font = font

# 单元格边框
borders = xlwt.Borders()
borders.top = xlwt.Borders.THICK
borders.bottom = xlwt.Borders.MEDIUM
borders.left = xlwt.Borders.MEDIUM
borders.right = xlwt.Borders.THICK
style5.borders = borders

# 对齐
alignment = xlwt.Alignment()
# 水平方向
alignment.horz = xlwt.Alignment.HORZ_CENTER
# 垂直方向
alignment.vert = xlwt.Alignment.VERT_CENTER
style5.alignment = alignment

In [111]:
# 编号样式
style6 = xlwt.XFStyle()

font = xlwt.Font()
font.name = '宋体'
font.bold = False
# 前一个数字是字号
font.height = 20*20 
# 样式复制
style6.font = font



# 对齐
alignment = xlwt.Alignment()
# 水平方向
alignment.horz = xlwt.Alignment.HORZ_CENTER
# 垂直方向
alignment.vert = xlwt.Alignment.VERT_CENTER
style6.alignment = alignment


# 写函数

In [112]:
# sheet_num sheet是第几个
# khyh 表示开户银行位于数据框的位置值
# zanghao 同理
# jinge
def add_new_sheet_about_kemuyue(sheet_num,khyh,zhanghao,jinge):
            
    new_sheet_kmye = new_excel.get_sheet(sheet_num)
    
    #试试看去掉页眉页脚
    new_sheet_kmye.header_str = b''
    new_sheet_kmye.footer_str = b''
    
    new_sheet_kmye.write(2,0,'开户银行：'+khyh,style0)
    new_sheet_kmye.write(2,13,'账号：'+zhanghao,style0)
    
    time.sleep(1)
    
    if len(jinge)>11:
        print(shiyan1.iloc[sheet_num,:])
    else:  
        for i in range(len(jinge)):
            new_sheet_kmye.write(5,12-i,jinge[-i-1],stylelist[i])

        for i in range(len(jinge)):
            new_sheet_kmye.write(5,25-i,jinge[-i-1],stylelist[i])
        
        for i in range(len(jinge)):
            new_sheet_kmye.write(37,12-i,jinge[-i-1],stylelist1[i])

        for i in range(len(jinge)):
            new_sheet_kmye.write(37,25-i,jinge[-i-1],stylelist1[i])
    
    new_sheet_kmye.write(3,25,'金        额',style5)
    new_sheet_kmye.write(3,12,'金        额',style5)
  
    new_sheet_kmye.write(1,25,sheet_num+1,style6)
    time.sleep(1)

# 调试前5个

In [113]:
# for j in range(0,5):
#     add_new_sheet_about_kemuyue(j,shiyan1.iloc[j,1],shiyan1.iloc[j,3],shiyan1.iloc[j,2])

# 文档保存地址改一下

In [114]:
# new_excel.save(r"格式.xls")

# 正式实现 ——运行前删除 格式.xls

In [115]:
for j in range(0,shiyan1.shape[0]):
    add_new_sheet_about_kemuyue(j,shiyan1.iloc[j,1],shiyan1.iloc[j,3],shiyan1.iloc[j,2])

IndexError: list index out of range

# 文档保存地址改一下

In [116]:
new_excel.save(r"格式1.xls")